In [1]:
# !pip3 install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
# !pip3 install gspread oauth2client
# !pip3 install requests
# !pip3 install langchain
# !pip3 install openai
# !pip3 install gspread oauth2client google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
# !pip3 install python-dotenv


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.4 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 182 kB 16.7 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 14.6 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 22.1 MB/s eta 0:00:01
     |████████████████████████████████| 224 kB 25.9 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 7.3 MB/s  eta 0:00:01
     |████████████████████████████████| 400 kB 20.3 MB/s eta 0:00:01
     |████████████████████████████████| 143 kB 27.4 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 18.4 MB/s eta 0:00:01
     |████████████████████████████████| 102 kB 19.8 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 9.2 MB/s  eta 0:00:01
     |████████████████████████████████| 122 kB 20.0 MB/s eta 0:00:01
  Using cached idna-3.4-py3-none-any.whl (61 kB)
     |████████████████████████████████| 156 kB 2

In [2]:
# for the predictions
import os
import sys
import sqlite3
import pathlib
import pandas as pd

# for AI Reg Regan
from langchain.llms import OpenAI
import openai

# for Google things
from dotenv import load_dotenv
import base64
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import gspread
from oauth2client.service_account import ServiceAccountCredentials

sys.path.append("use-predictions") 
import save_predictions as sp

# Get to the root directory
project_root = pathlib.Path().absolute().parent.parent

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"
secrets_path = project_root / "secrets.env"

# lead secrets
load_dotenv(dotenv_path=secrets_path)

True

In [3]:
# Connect to the SQLite database
con = sqlite3.connect(str(db_path))

# Query to select all records from the predictions_table
query = """
    WITH min_round_id AS (
        SELECT MIN(round_id) AS round_id
        FROM footy_tipping_data
        WHERE game_state_name = 'Pre Game'
    )

    SELECT ft.game_id
        , p.home_team_result
        , ft.team_home
        , ft.position_home
        , ft.team_head_to_head_odds_home
        , ft.team_away
        , ft.position_away
        , ft.team_head_to_head_odds_away
        , p.home_team_win_prob
        , p.home_team_lose_prob
        , ft.home_elo
        , ft.away_elo
        , ft.round_id
        , ft.competition_year
        , ft.round_name
    FROM predictions_table p
    LEFT JOIN footy_tipping_data ft ON p.game_id = ft.game_id
    WHERE ft.game_state_name = 'Pre Game'
    AND round_id = (SELECT * FROM min_round_id)
"""

# Execute the query and fetch the results into a data frame
predictions = pd.read_sql_query(query, con)

# Disconnect from the SQLite database
con.close()

# Get prod_run from environment variable. Convert it to boolean.
prod_run = os.getenv('PROD_RUN', 'False') == 'True'

sp.save_predictions(predictions, prod_run=prod_run)
tipper_picks = sp.tipper_picks(predictions, prod_run=prod_run)

predictions

,game_id,home_team_result,team_home,position_home,team_head_to_head_odds_home,team_away,position_away,team_head_to_head_odds_away,home_team_win_prob,home_team_lose_prob,home_elo,away_elo,round_id,competition_year,round_name,home_odds_thresh,away_odds_thresh
0,2.023111e+10,Loss,St. George Illawarra Dragons,15.0,2.75,New Zealand Warriors,6.0,1.45,0.435300,0.564700,1486.458681,1509.067023,17.0,2023.0,Round 17,2.297266,1.770852
1,2.023111e+10,Loss,Dolphins,10.0,2.75,Parramatta Eels,8.0,1.45,0.393582,0.606418,1477.222338,1538.373525,17.0,2023.0,Round 17,2.540769,1.649027
2,2.023111e+10,Win,Penrith Panthers,2.0,1.55,Newcastle Knights,14.0,2.45,0.591117,0.408883,1543.327663,1487.508067,17.0,2023.0,Round 17,1.691713,2.445687
3,2.023111e+10,Win,Melbourne Storm,3.0,1.20,Manly-Warringah Sea Eagles,12.0,4.50,0.627509,0.372491,1529.594162,1494.579544,17.0,2023.0,Round 17,1.593604,2.684625
4,2.023111e+10,Win,Brisbane Broncos,1.0,1.35,Gold Coast Titans,9.0,3.25,0.626871,0.373129,1517.723248,1485.104398,17.0,2023.0,Round 17,1.595224,2.680041
5,2.023111e+10,Win,South Sydney Rabbitohs,4.0,1.45,North Queensland Cowboys,13.0,2.75,0.595493,0.404507,1524.943235,1498.624605,17.0,2023.0,Round 17,1.679280,2.472146
6,2.023111e+10,Win,Sydney Roosters,11.0,1.60,Canberra Raiders,7.0,2.35,0.554381,0.445619,1481.271216,1484.287857,17.0,2023.0,Round 17,1.803815,2.244067


# Langchain

In [6]:
llm = OpenAI(openai_api_key=os.getenv('OPENAI_KEY'),
             model_name="gpt-3.5-turbo-16k",
             max_tokens=15000,
             temperature=1.1)

input_predictions = ""

for index, row in predictions.iterrows():
    input_predictions += f"""

        Round Name: {row['round_name']},
        Home Team Result: {row['home_team_result']},
        Home Team: {row['team_home']}, 
        Home Team Position: {row['position_home']},
        Home Team Head to Head Price: {row['team_head_to_head_odds_home']}
        Away Team: {row['team_away']},
        Away Team Position: {row['position_away']},
        Away Team Head to Head Price: {row['team_head_to_head_odds_away']}\n

        """
    
prompt = f"""

    I have a set of predictions for NRL games in {predictions['round_name'].unique()[0]} {predictions['competition_year'].unique()[0]} made by a machine learning pipeline called the Footy Tipper: \n{input_predictions}\n 

    The description of the columns of interest is:

    * Home Team Result: the predicted result of the home team
    * Home Team: the home team
    * Home Team Position: the home team's position on the NRL ladder
    * Home Team Head to Head Price: the price bookies are offering for a home win
    * Away Team: the away team
    * Away Team Position: the away team's position on the NRL ladder
    * Away Team Head to Head Price: the price bookies are offering for an away win

    Could you write up an email to my mates from Reg Regan, giving them a synopsis of the round along withthe tips? 
    Accompany the tips with some smart arsed comments to about the teams playing.
    
    Remember to link everyone to the tips folder: https://drive.google.com/drive/folders/1lT1SQH-kQPdnOf3ftomobuiv51jEj6t-?usp=sharing
    Also, tell everyone to bring back the biff at the end of the email.

    Always sign off the email as Reg Regan.
    
    """

reg_regan = llm.predict(prompt)

print(reg_regan)

Dear mates,

Hope you're all doing well. I wanted to give you a heads up on the upcoming NRL Round 17 games and share some predictions made by a nifty machine learning pipeline called the Footy Tipper. As always, here's the link to the tip folder for your reference: [NRL Round 17 Tips](https://drive.google.com/drive/folders/1lT1SQH-kQPdnOf3ftomobuiv51jEj6t-?usp=sharing).

Let's dive into the predictions and add a touch of smart arsed commentary for good measure:

1. St. George Illawarra Dragons vs New Zealand Warriors: The Footy Tipper foresees a loss for the Dragons. When you have mischievous Warriors on the field, dragons tend to get roasted. Keep the fire extinguisher nearby. 

2. Dolphins vs Parramatta Eels: The Dolphins are predicted to lose in this match. Looks like the Eels are in no mood to play nice-nice with the newbies. Swimming lessons might be needed after this game.

3. Penrith Panthers vs Newcastle Knights: The mighty Penrith Panthers are expected to triumph. Even knight

# send emails

In [5]:
project_root = pathlib.Path().absolute().parent.parent
json_path = project_root / "footy-tipper-c5bcb9639ee2.json"

# Use your downloaded credentials file
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
client = gspread.authorize(creds)

# Open the test sheet and get the data
sheet = client.open("footy-tipper-email-list").sheet1  # use your actual sheet name
email_data = sheet.get_all_records()  # gets all the data inside your Google Sheet

your_mates_emails = [row['Email'] for row in email_data]  # replace 'Email' with your actual column name


In [6]:
# Setup the email
msg = MIMEMultipart()
msg['From'] = os.getenv('MY_EMAIL')
msg['To'] = ', '.join(your_mates_emails)
msg['Subject'] = f"Footy Tipper Predictions for {predictions['round_name'].unique()[0]}"

# Add your message
message = reg_regan  # or whatever variable has your final message
msg.attach(MIMEText(message, 'plain'))

# Setup the SMTP server
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()

# Add your credentials
server.login(os.getenv('MY_EMAIL'), os.getenv('EMAIL_PASSWORD'))

# Send the email
text = msg.as_string()
server.sendmail(os.getenv('MY_EMAIL'), your_mates_emails, text)

# Close the connection
server.quit()

(221,
 b'2.0.0 closing connection p38-20020a634f66000000b005535ddd8dcfsm5431199pgl.89 - gsmtp')